Copyright (c) 2022, salesforce.com, inc and MILA.  
All rights reserved.  
SPDX-License-Identifier: BSD-3-Clause  
For full license text, see the LICENSE file in the repo root  
or https://opensource.org/licenses/BSD-3-Clause  

In [1]:
!python --version

Python 3.7.16


In [3]:
import os

In [1]:
import torch
torch.__version__

'1.9.0+cu102'

In [1]:
import logging
import numpy as np
import os
import shutil
import subprocess
import sys
import time
import yaml

sys.path.append(os.getcwd() + '/scripts')

#os.chdir(os.getcwd() + "/scripts")
#from scripts.train_with_rllib import create_trainer, fetch_episode_states, load_model_checkpoints
#os.chdir("..")
#from scripts import fixed_paths, evaluate_submission, run_unittests, train_with_rllib
from scripts.train_with_rllib import create_trainer, fetch_episode_states, load_model_checkpoints

/home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Set logger level e.g., DEBUG, INFO, WARNING, ERROR.
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# Needed to perform this install when the system threw the lib.so file missing error
# ! apt-get install libglib2.0-0 --yes

In [3]:
print("Training with RLlib...")
# Read the run configurations specific to the environment.
# Note: The run config yaml(s) can be edited at warp_drive/training/run_configs
# -----------------------------------------------------------------------------
config_path = os.path.join("scripts", "rice_rllib.yaml")
if not os.path.exists(config_path):
    raise ValueError(
        "The run configuration is missing. Please make sure the correct path"
        "is specified."
    )

with open(config_path, "r", encoding="utf8") as fp:
    run_config = yaml.safe_load(fp)

# Create trainer
# --------------
trainer, save_dir = create_trainer(run_config)
# Copy the source files into the results directory
# ------------------------------------------------
os.makedirs(save_dir)
for file in [
    "rice.py",
]:
    shutil.copyfile(
        os.path.join(file),
        os.path.join(save_dir, file),
    )
    
# Add an identifier file
with open(os.path.join(save_dir, ".rllib"), "x", encoding="utf-8") as fp:
    pass
fp.close()

Training with RLlib...


2023-03-21 18:00:17,936	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2023-03-21 18:00:19,311	INFO trainer.py:618 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=29559) /home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=29559)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=29559) /home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=29559)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=29559) /home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

2023-03-21 18:00:20,500	WARNING util.py:39 -- Install gputil for GPU system monitoring.
(pid=29559) WARNING:tensorflow:From /home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=29559) Instructions for updating:
(pid=29559) non-resource variables are not supported in the long term
(pid=29561) WARNING:tensorflow:From /home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=29561) Instructions for updating:
(pid=29561) non-resource variables are not supported in the long term
(pid=29560) WARNING:tensorflow:From /home/ivaylo/.local/lib/python3.7/site-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope)

### Invoke training

In [6]:
trainer.config

{'num_workers': 4,
 'num_envs_per_worker': 5,
 'rollout_fragment_length': 20,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 2000,
 'model': {'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  '_time_major': False,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {'num_discrete_action_levels': 10,
  'negotiation_on': False,
  'source_dir': None},
 'env': 'EnvWrapper',
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preprocessor_pr

In [ ]:
NUM_ITERS = 5
for iter in range(NUM_ITERS):
    result = trainer.train()
print(result)

### Fetch episode states

In [ ]:
outputs = fetch_episode_states(trainer, ["T_i", "carbon_mass_i", "capital_i"])